# Basic CNN

Import libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

Define the model

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3) # CIFAR10 images are color (3 channels)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(2304,1024)
        self.fc2 = nn.Linear(1024,512)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch,
                batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss,
        correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))

Main execution

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
train_kwargs = {'batch_size': 500}
test_kwargs = {'batch_size': 500}
if use_cuda:
    cuda_kwargs = {'num_workers': 1, 'pin_memory': True, 'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
dataset1 = datasets.CIFAR10('../data', train=True, download=True, transform=transform)
dataset2 = datasets.CIFAR10('../data', train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = torch.optim.Adadelta(model.parameters(), lr=1.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(10):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Files already downloaded and verified
Train Epoch: 0 [0/50000 (0%)]	Loss: -0.007420
Train Epoch: 0 [10000/50000 (20%)]	Loss: -25295712256.000000
Train Epoch: 0 [20000/50000 (40%)]	Loss: -75158401546209722368.000000
Train Epoch: 0 [30000/50000 (60%)]	Loss: -20426487607223544370002132992.000000
Train Epoch: 0 [40000/50000 (80%)]	Loss: -206952875408483742973528375296.000000

Test set: Average loss: -385516569369010625330251038720.0000, Accuracy: 1000/10000 (10%)

Train Epoch: 1 [0/50000 (0%)]	Loss: -387502244232606040414048223232.000000
Train Epoch: 1 [10000/50000 (20%)]	Loss: -511240637573441794961924161536.000000
Train Epoch: 1 [20000/50000 (40%)]	Loss: -639022321096910539741766615040.000000
Train Epoch: 1 [30000/50000 (60%)]	Loss: -759375873258552784429112098816.000000
Train Epoch: 1 [40000/50000 (80%)]	Loss: -904649466687093735780616503296.000000

Test set: Average loss: -1015926727545403874351477948416.0000, Accuracy: 1000/10000 (10%)

Train Epoch: 2 [0/50000 (0%)]	Loss: -10192673458

KeyboardInterrupt: 

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

# Define the model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1) # CIFAR10 images are color (3 channels)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128) # Adjust input size based on pooling
        self.fc2 = nn.Linear(128, 10) # 10 output classes for CIFAR10

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Data loading and preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False)

# Training
model = Net().to(device)
optimizer = torch.optim.Adadelta(model.parameters(), lr=1.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(10):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Files already downloaded and verified
Files already downloaded and verified
Train Epoch: 0 [0/50000 (0%)]	Loss: -0.012847
Train Epoch: 0 [10000/50000 (20%)]	Loss: -4116340224.000000
Train Epoch: 0 [20000/50000 (40%)]	Loss: -11634631822893121536.000000
Train Epoch: 0 [30000/50000 (60%)]	Loss: -4354254473755094217054486528.000000
Train Epoch: 0 [40000/50000 (80%)]	Loss: -342532054910603120015902244864.000000

Test set: Average loss: -765223845909978957832576303104.0000, Accuracy: 1000/10000 (10%)

Train Epoch: 1 [0/50000 (0%)]	Loss: -773873009455916513834894884864.000000
Train Epoch: 1 [10000/50000 (20%)]	Loss: -1075664944266280163892109770752.000000
Train Epoch: 1 [20000/50000 (40%)]	Loss: -1368363036830047669639891124224.000000
Train Epoch: 1 [30000/50000 (60%)]	Loss: -1662264237257922909159475380224.000000
Train Epoch: 1 [40000/50000 (80%)]	Loss: -1984032084522280062127918022656.000000

Test set: Average loss: -2290806495272547058198488547328.0000, Accuracy: 1000/10000 (10%)

Train Ep

In [5]:
conda install matplotlib

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\anaconda3

  added / updated specs:
    - matplotlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    brotli-1.0.9               |       h2bbff1b_8          19 KB
    brotli-bin-1.0.9           |       h2bbff1b_8          29 KB
    contourpy-1.2.0            |  py310h59b6b97_0         203 KB
    fonttools-4.51.0           |  py310h2bbff1b_0         2.0 MB
    kiwisolver-1.4.4           |  py310hd77b12b_0          60 KB
    libbrotlicommon-1.0.9      |       h2bbff1b_8          79 KB
    libbrotlidec-1.0.9         |       h2bbff1b_8          39 KB
    libbrotlienc-1.0.9         |       h2bbff1b_8         261 KB
    matplotlib-3.8.4           |  py310haa95532_0           9 KB
    matplotlib-base-3.8.4      |  py310h4ed8f06_0         8.7 MB
    pyparsing-3.0.9            |  py310haa95532_0  



==> WARNING: A newer version of conda exists. <==
  current version: 23.5.2
  latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.5.0


